<a href="https://colab.research.google.com/github/Echo9k/Information-Retrival/blob/main/PA7_part4_WebCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web crawler
Hello there!.

I made a quick adaptation of the steamer from the [ notebook previous](https://colab.research.google.com/drive/14AV2jstGnxw67HaK7o6tZfAun0zVGx_s)

You're taking a look into brand new crawling code inspired (not copied) from Mitchell, (2018) [original source code 3](https://colab.research.google.com/github/REMitchell/python-scraping/blob/master/Chapter03-web-crawlers.ipynb) )


Mitchell, R. (2018). Web Scraping with Python: Collecting More Data from the Modern Web (2nd ed.). O’Reilly Media.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://rebrand.ly/PA7_part4_WebCrawler_colab"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://rebrand.ly/PA7_part4_WebCrawler_git"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://rebrand.ly/PA7_part4_WebCrawler_notebook"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>

  </td>
  <td>
    <a target="_blank" href="https://rebrand.ly/PA7_part4_WebCrawler_py"><img src="https://miro.medium.com/max/256/1*ztqS5rRI29GHxZa6uPF2UA.png" width="40" height="40" />Download .py</a>
  </td>
</table>

## Set up

In [1]:
#@markdown ### Import libriaries

# Other
import sys, os, re, time, timeit
import tqdm
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from fastprogress import master_bar, progress_bar

# Page processing
from urllib.request import urlopen 
from urllib.error import URLError
from bs4 import BeautifulSoup 

# DataFrames
import pandas as pd
from google.colab.data_table import DataTable
%load_ext google.colab.data_table

In [2]:
#@title Class Crawler
# crawler
class Crawler():
    def __init__(self, domain):
        self.domain = domain
        self.queue=[domain]
        self.text = {}  # Text just for the first few sites.
    
    def make_soup(self, URL)->BeautifulSoup:
        try:
            html = urlopen(URL)
            return BeautifulSoup(html, 'html.parser')
        except (TimeoutError, URLError) as err:
            print(f"error: {err} on {URL}")
            raise

    def get_urls(self, URL, include_external=False)-> (set, str):
        soup=self.make_soup(URL)
        regx_internal= re.compile('^(?!#)((?!\/\/).)*$')  # Incomplete links are internal
        regx_full_URL= re.compile('^(ht)(.)*$')

        internal=set([self.domain+link.attrs['href'] for link in soup.find_all('a', href=regx_internal) if 'href' in link.attrs])
        if include_external:
            full_URLs=set([link.attrs['href'] for link in soup.find_all('a', href=regx_full_URL) if 'href' in link.attrs])
            links = internal.union(full_URLs)
        else:
            links = internal
        return links, soup.text

    def queue_maker(self, queue_limit=500, quiet=False):
        while len(self.queue)<=queue_limit:
            URL = self.queue.pop(0)
            try: links, text = self.get_urls(URL)
            except (TimeoutError, URLError) as err: continue
            except: print(f"Unexpected error: {sys.exc_info()[0]}  while visiting {URL}"); continue
            self.text.update({URL: text})
            if links is not None:
                new_unique_URLs=set(self.queue).union(self.text)\
                                .symmetric_difference(links).intersection(links)
                self.queue.extend(new_unique_URLs)
                if not quiet: print(f"Queue size: {len(self.queue):<7} {len(new_unique_URLs):> 3} new unique URLs from  {URL: <7}")
        return self.queue, self.text

    def extractor(self):
        """
        
        """
        for URL in progress_bar(self.queue):
            # From the URL the soup>text 
            try:soup=self.make_soup(URL)
            except: print(f"Unexpected error: {sys.exc_info()[0]}  while visiting {URL}"); continue
            
            # Add text to dictionary
            self.text.update({URL: soup.get_text(strip=True)})

# Extract the info

In [3]:
domain='http://www.hometownlife.com'  #@param{type:"string"}
limit_queue = 500 #@param {type:"slider", min:50, max:550, step:50}
crawler = Crawler(domain)
crawler.queue_maker(limit_queue, quiet=True)
print("extracting...")
crawler.extractor()

#@markdown Extracted text stored on raw_data.csv with this data format: (url, raw text)


use_sample = True #@param {type:"boolean"}
#@markdown > Due to the size of the queue  it's adviced to use a sample.

data = pd.Series(crawler.text)
data.index.name = "url"
data.name = "text"
data.to_csv("raw_data.csv")

if use_sample: data=data[:20]

Queue size: 32       32 new unique URLs from  http://www.hometownlife.com
Queue size: 37        6 new unique URLs from  http://www.hometownlife.com/story/sports/high-school/2021/03/15/michigan-high-school-hockey-playoff-scoreboard/4700588001/
Queue size: 41        5 new unique URLs from  http://www.hometownlife.com/story/life/community/observer/livonia/2021/03/18/livonia-native-takes-job-deputy-press-secretary-biden-white-house/4706522001/
Queue size: 50       10 new unique URLs from  http://www.hometownlife.com/story/news/2021/03/17/here-public-records-requests-local-schools-received-2020/4714351001/
Queue size: 54        5 new unique URLs from  http://www.hometownlife.com/story/entertainment/2021/03/05/livonia-native-churchill-alum-appear-jeopardy-friday/4594297001/
Queue size: 145      92 new unique URLs from  http://www.hometownlife.com/sitemap/
Queue size: 150       6 new unique URLs from  http://www.hometownlife.com/story/life/community/hometown/novi/2021/03/17/1-billion-mega-mil

error: <urlopen error [Errno -2] Name or service not known> on http://www.hometownlife.commailto:?subject=%241%20billion%20Mega%20Millions%20winners%20burned%20breakfast%2C%20then%20called%20a%20lottery%20lawyer%20-%20from%20Observer%20and%20Eccentric%20Newspapers%20and%20Hometown%20Weeklies&body=%241%20billion%20Mega%20Millions%20winners%20burned%20breakfast%2C%20then%20called%20a%20lottery%20lawyer%0A%0AKurt%20Panouses%20shares%20the%20tips%20he%20gives%20newly%20minted%20millionaires%2C%20including%20the%20Michigan%20ones%20who%20just%20won%20a%20billion.%0A%0ACheck%20out%20this%20story%20on%20hometownlife.com%3A%20https%3A%2F%2Fwww.hometownlife.com%2Fstory%2Flife%2Fcommunity%2Fhometown%2Fnovi%2F2021%2F03%2F17%2F1-billion-mega-millions-winners-burned-breakfast-called-lawyer%2F4706879001%2F
Unexpected error: <class 'urllib.error.URLError'>  while visiting http://www.hometownlife.commailto:?subject=%241%20billion%20Mega%20Millions%20winners%20burned%20breakfast%2C%20then%20called%20a%

# Steamer

## Setup

In [4]:
#@title installing libraries
%%capture
%%bash
pip install -U textblob
python -m textblob.download_corpora
rm /content/sample_data -r

In [5]:
#@title Imports
#@markdown Import the necesary libraries for this notebook to work.
%%capture
# Numerical computations
import scipy.stats as stats
import numpy as np
import math

from sklearn import preprocessing
from sklearn import metrics
from collections import Counter

# Graphs
import seaborn as sns
import plotly.express as px
# Read HTML
import codecs
from bs4 import BeautifulSoup as bs
# Text processing
from textblob import TextBlob
from textblob import Word
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# from nltk.tokenize import word_tokenize  
# from nltk.stem import PorterStemmer

In [6]:
#@title Stopwatch functionality
#@markdown Wrapper function from [realPython](https://realpython.com/primer-on-python-decorators/)
import functools
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

## Text processing

In [7]:
#@title Steamer class
#@markdown 1. **Stop words removal:** Removes english stopwords from the sentence.<br><br>
#@markdown 2. **Language:** Select the language of your text so we know the correct stopwords.
language = "spanish" #@param ["turkish", "spanish", "english", "french", "german", "swedish"]

#@markdown 3. **Method:** Applies one of the following types of text normalization.
#@markdown * Lematize: Gets the core/regular form of each words.
#@markdown * Steam: Truncates each word by removing es/ing/...
#@markdown * Remove stopwords: Returns the stopwords witouth any change.<br><br>
method = "steam" #@param ["steam", "lematize", "Remoe stopwords"]


class corpora_processor():
    def __init__(self, method, language="english"):
        self.method = method
        self.stopwords = stopwords.words(language)
        self.sw_count = 0
        self.numeric_tokens = 0
        self.small_tokens = 0
    def text_processor(self, text) -> list:
        def _process(word):
            # Skips numeric tokens
            try:
                int(word)                
                self.numeric_tokens += 1
                return None
            except ValueError:
                pass
            # Skips stop words
            if word in self.stopwords:
                self.sw_count += 1
                return None
            # Skips small tokens
            if len(word)<3:
                self.small_tokens += 1
                return None
                
            else:
                if self.method == "lematize": return word.lemmatize("v")
                if self.method == "steam": return word.stem()
                else: return word
            

        words = TextBlob(text).words
        words = words.lower()
        
        return [_process(word) for word in words if _process(word) is not None]

processor = corpora_processor(method, language)
sample_text = "Esto es un 1111 texto de prueba el investigador que este estudiando el tema o algun estudiante para el estudiante" #@param {type:"string"}
processor.text_processor(sample_text)

['texto',
 'prueba',
 'investigador',
 'estudiando',
 'tema',
 'algun',
 'estudiant',
 'estudiant']

## Indexing
Let's process the corpus but this time applaying removing stopwords and applaying text normalization.

Ignore from the index
* Stop words for the selected language
* Tokens with less than 2 characters in length
* Numerical terms
* Starting with a punctuation character are not indexed

### Text processing
Integrate the Porter Stemmer code into our index

### Additioanl (new) metrics
Calculate the tf-idft,d value for each unique combination of document and term


In [8]:
#@title Text processing function
#@markdown This function returns:

@timer
def indexer(data:dir, processor, *,output_path:str="./"):
    total_tokens = 0
    total_documents = 0
    token_count = {}  # Term frequency corpus
    token_document = {} # Token:[document which contains it]
    label_encoder = preprocessing.LabelEncoder()
    # document_list = [os.path.join(dirname, i) for i in os.listdir(dirname) if os.path.isfile(os.path.join(dirname, i))]
    tfd = {}
    tf_idf = {}
    for url, text in progress_bar(data.items()):
        total_documents += 1
        tf_idf.update({url:{}})
        tfd.update({url:{}})
        local_tokens = processor.text_processor(text)
        N = len(local_tokens)

        # Metrics & data structures
        total_tokens += N
        
        for t in local_tokens:
            # Term frecuency in the corpus
            try: token_count[t] += 1
            except: token_count[t] = 1
            # Frecuency of a token per document document:tf
            try: tfd[url][t] += 1
            except KeyError: tfd[url].update({t:1})
            # The documents where you find a token: {token:[document]}
            try: token_document[t].append(url)
            except KeyError: token_document.update({t:[url]})
        
        # TF_IDF
        for t in set(local_tokens):
            # Calculate the Inverse document frequency: idf
            idf = np.log(N/tfd[url][t])
            tf =  np.log(1+token_count[t])
            tf_idf[url].update({t:tf*idf})

        # Progressbar
        # sys.stdout.write("-")
        # sys.stdout.flush()
        # if enum%100==0: print(f"\n{enum/len(data)} [{enum}/{len(data)}]")


    # return to original directory
    os.chdir(output_path)


    # Write to CSV
    processed_documents = sorted(tfd.keys())
    processed_documents = pd.DataFrame(processed_documents)
    processed_documents.to_csv("processed_documents_df.csv")
    
    index_encoded = label_encoder.fit_transform(list(token_count.keys()))
    term_frequency_corpus = pd.DataFrame.from_dict(token_count,
                                            orient='index',
                                            columns=['count'])\
                                .sort_values('count')
    term_frequency_corpus.to_csv("token_count.csv")

    #output
    metrics = {
        "total_documents": total_documents,
        "total_tokens": total_tokens,
        "unique_tokens":len(token_count),
        "stopword_count":processor.sw_count,
        "numeric_tokens":processor.numeric_tokens,
        "small_tokens":processor.small_tokens
        }
    data_structures = {        
        "term_frequency_corpus": term_frequency_corpus,
        "token_document":token_document,
        "processed_documents":processed_documents,
        "label_encoder":label_encoder
        }
    scores = {        
        "tfd": tfd,
        "idf":{k: np.log(total_tokens/v) for k,v in token_count.items()},
        "tf_idf":tf_idf,
        "token count":token_count
        }
    return metrics, data_structures, scores
#@markdown Metrics
#@markdown * total_documents: Count of documents
#@markdown * total_tokens: Count of tokens processed.
#@markdown * unique_tokens: Unique tokens processed.
#@markdown * stopword_count: Count of stopwords.
#@markdown > Ignores irrelevant tokens _(Small, numeric, stopwords)_

#@markdown Data Structures
#@markdown * term_frequency_corpus: Frequency of each term in the corpus.
#@markdown * token_document: In which documents you find what token
#@markdown * processed_documents: List of processed documents


#@markdown Scores
#@markdown * tfd: Term frequency by document
#@markdown * inverse_document_frequency: Inverse term frequency by document

In [9]:
#@title Build the index
#@markdown Let's recreate the index and take a meassure of the new metrics.

#@markdown ### Configure the text function
method = "steam" #@param ["steam", "lematize", "Remoe stopwords"]
language = "english" #@param ["turkish", "spanish", "english", "french", "german", "swedish"]
processor = corpora_processor(method, language)
metrics, data_structures, scores = indexer(data.to_dict(), processor)

print(
    f"Total documents:\t{metrics['total_documents']}\n"
    f"Total tokens:\t\t{metrics['total_tokens']} \n"
    f"Unique tokens:\t\t{metrics['unique_tokens']}\n"
    f"stopword count: \t{metrics['stopword_count']}\n"
    # f"Small tokens:\t\t{metrics['small_tokens']}\n"
    # f"Numeric tokens:\t\t{metrics['numeric_tokens']}\n"
    
)

Finished 'indexer' in 71.7597 secs
Total documents:	20
Total tokens:		126411 
Unique tokens:		7191
stopword count: 	15626



# Statistics
The order in which the URLs are stored varies a little but this is the result from one full execution. In the above you see only a partial result from 20 URL because the parsing of the whole 800 [^1]


|metric|value|
|:---|----|
Number of documents processed | 527
Total number of terms parsed from all documents | 1715640
Total number of unique terms found and added to the index | 35911
Total number of terms found that matched one of the stop words in your program’s stop words list | 263241


[^1]: yes, 800. It jumped from about 450 to 800 when it grabed the sitemap from 2014 which is the expected behaviour whith my design.

```
while queue_length <=500:
    # add all from this page
```

In [14]:
#@title Term frequency
#@markdown Let's take a look on the resulting term frequencies:
data_structures['term_frequency_corpus']

,count
www.gannett-cdn.com/presto/2021/03/15/poen/0bb51199-03dd-4b9b-9d86-fb449db2299c-freedompassover3.jpg,1
min-height:106px,1
padding-right:200px,1
gnt_se_a_f,1
width:188px,1
...,...
bidder,1131
color,1150
posit,1356
none,1509
